meshcnn, pointnet, 

In [28]:
import trimesh
import numpy as np

import meshplot as mp
import torch
from pathlib import Path
from torch_geometric.data import data
from torch_geometric.loader import DataLoader



In [3]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [4]:
# https://colab.research.google.com/drive/1Uds5r4QWcB5bsELiyBM8TBHEHFVCVcd3?usp=sharing#scrollTo=gr6HXz0qqSxd
# https://medium.com/stanford-cs224w/deep-learning-on-3d-meshes-9608a5b33c98

def load_mesh(mesh_filename: Path):
    """Extract vertices and faces from raw mesh file.

    Parameters
    ----------
    mesh_filename: PathLike
        Path to mesh `.ply` file.

    Returns
    -------
    vertices: torch.tensor
        Float tensor of size (|V|, 3), where each row
        specifies the spatial position of a vertex in 3D space.
    faces: torch.tensor
        Intger tensor of size (|M|, 3), where each row
        defines a traingular face.
    """
    mesh = trimesh.load_mesh(mesh_filename, process=False)
    vertices = torch.from_numpy(mesh.vertices).to(torch.float)
    faces = torch.from_numpy(mesh.faces)
    faces = faces.t().to(torch.long).contiguous()
    return vertices, faces

In [27]:
mesh = trimesh.load_mesh("Meshes/cab4.stl")
mesh

<trimesh.Trimesh(vertices.shape=(9200, 3), faces.shape=(18420, 3))>

In [24]:
mesh.show()

In [31]:
from torch_geometric.utils import from_trimesh



In [33]:
newmesh = from_trimesh(mesh)
newmesh

Data(pos=[9200, 3], face=[3, 18420])

In [30]:
import glob
import os
import os.path as osp
from typing import Callable, List, Optional

import torch

from torch_geometric.data import InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_off

class GeometricShapes(InMemoryDataset):
    r"""Synthetic dataset of various geometric shapes like cubes, spheres or
    pyramids.

    .. note::

        Data objects hold mesh faces instead of edge indices.
        To convert the mesh to a graph, use the
        :obj:`torch_geometric.transforms.FaceToEdge` as :obj:`pre_transform`.
        To convert the mesh to a point cloud, use the
        :obj:`torch_geometric.transforms.SamplePoints` as :obj:`transform` to
        sample a fixed number of points on the mesh faces according to their
        face area.

    Args:
        root (str): Root directory where the dataset should be saved.
        train (bool, optional): If :obj:`True`, loads the training dataset,
            otherwise the test dataset. (default: :obj:`True`)
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
        pre_filter (callable, optional): A function that takes in an
            :obj:`torch_geometric.data.Data` object and returns a boolean
            value, indicating whether the data object should be included in the
            final dataset. (default: :obj:`None`)

    **STATS:**

    .. list-table::
        :widths: 10 10 10 10 10
        :header-rows: 1

        * - #graphs
          - #nodes
          - #edges
          - #features
          - #classes
        * - 80
          - ~148.8
          - ~859.5
          - 3
          - 40
    """

    url = 'https://github.com/Yannick-S/geometric_shapes/raw/master/raw.zip'

    def __init__(self, root: str, train: bool = True,
                 transform: Optional[Callable] = None,
                 pre_transform: Optional[Callable] = None,
                 pre_filter: Optional[Callable] = None):
        super().__init__(root, transform, pre_transform, pre_filter)
        path = self.processed_paths[0] if train else self.processed_paths[1]
        self.data, self.slices = torch.load(path)

    @property
    def raw_file_names(self) -> str:
        return '2d_circle'

    @property
    def processed_file_names(self) -> List[str]:
        return ['training.pt', 'test.pt']

    def download(self):
        path = download_url(self.url, self.root)
        extract_zip(path, self.root)
        os.unlink(path)

    def process(self):
        torch.save(self.process_set('train'), self.processed_paths[0])
        torch.save(self.process_set('test'), self.processed_paths[1])

    def process_set(self, dataset: str):
        categories = glob.glob(osp.join(self.raw_dir, '*', ''))
        categories = sorted([x.split(os.sep)[-2] for x in categories])

        data_list = []
        for target, category in enumerate(categories):
            folder = osp.join(self.raw_dir, category, dataset)
            paths = glob.glob(f'{folder}/*.off')
            for path in paths:
                data = read_off(path)
                data.pos = data.pos - data.pos.mean(dim=0, keepdim=True)
                data.y = torch.tensor([target])
                data_list.append(data)

        if self.pre_filter is not None:
            data_list = [d for d in data_list if self.pre_filter(d)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(d) for d in data_list]

        return self.collate(data_list)

In [ ]:
class SegmentationFaust(InMemoryDataset):
    map_seg_label_to_id = dict(
        head=0,
        torso=1,
        left_arm=2,
        left_hand=3,
        right_arm=4,
        right_hand=5,
        left_upper_leg=6,
        left_lower_leg=7,
        left_foot=8,
        right_upper_leg=9,
        right_lower_leg=10,
        right_foot=11,
    )

    def __init__(self, root, train: bool = True, pre_transform=None):
        """
        Parameters
        ----------
        root: PathLike
            Root directory where the dataset should be saved.
        train: bool
            Whether to load training data or test data.
        pre_transform: Optional[Callable]
            A function that takes in a torch_geometric.data.Data object
            and outputs a transformed version. Note that the transformed
            data object will be saved to disk.

        """
        super().__init__(root, pre_transform)
        path = self.processed_paths[0] if train else self.processed_paths[1]
        self.data, self.slices = torch.load(path)

    @property
    def processed_file_names(self) -> list:
        return ["training.pt", "test.pt"]

    @property
    @lru_cache(maxsize=32)
    def _segmentation_labels(self):
        """Extract segmentation labels."""
        path_to_labels = Path(self.root) / "MPI-FAUST"/ "segmentations.npz"
        seg_labels = np.load(str(path_to_labels))["segmentation_labels"]
        return torch.from_numpy(seg_labels).type(torch.int64)

    def _mesh_filenames(self):
        """Extract all mesh filenames."""
        path_to_meshes = Path(self.root)/ "MPI-FAUST" / "meshes"
        return path_to_meshes.glob("*.ply")

    def _unzip_dataset(self):
        """Extract dataset from zip."""
        path_to_zip = Path(self.root) / "MPI-FAUST.zip"
        extract_zip(str(path_to_zip), self.root, log=False)

    def process(self):
        """Process the raw meshes files and their corresponding class labels."""
        self._unzip_dataset()

        data_list = []
        for mesh_filename in sorted(self._mesh_filenames()):
            vertices, faces = load_mesh(mesh_filename)
            data = Data(x=vertices, face=faces)
            data.segmentation_labels = self._segmentation_labels
            if self.pre_transform is not None:
                data = self.pre_transform(data)
            data_list.append(data)

        torch.save(self.collate(data_list[:80]), self.processed_paths[0])
        torch.save(self.collate(data_list[80:]), self.processed_paths[1])

In [26]:
first_cab1 = load_mesh("Meshes/cab4.stl")
first_cab1

(tensor([[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.7812,  0.0000],
         [ 0.4933,  1.5970,  0.0000],
         ...,
         [ 0.4659, 11.2611,  0.2767],
         [ 0.4933, 11.2655,  0.2766],
         [ 0.4659, 11.2655,  0.2766]]),
 tensor([[    0,     3,     6,  ..., 55251, 55254, 55257],
         [    1,     4,     7,  ..., 55252, 55255, 55258],
         [    2,     5,     8,  ..., 55253, 55256, 55259]]))

In [ ]:
cab3 = 

In [6]:
first_cab1

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.7812,  0.0000],
        [ 0.4933,  1.5970,  0.0000],
        ...,
        [ 0.4659, 11.2611,  0.2767],
        [ 0.4933, 11.2655,  0.2766],
        [ 0.4659, 11.2655,  0.2766]])

In [22]:
first_cab2

tensor([[    0,     3,     6,  ..., 55251, 55254, 55257],
        [    1,     4,     7,  ..., 55252, 55255, 55258],
        [    2,     5,     8,  ..., 55253, 55256, 55259]])

In [7]:
first_cab2.shape

torch.Size([3, 18420])

In [14]:
fc1 = first_cab1.numpy()
fc1

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.7812308 ,  0.        ],
       [ 0.49328175,  1.5969656 ,  0.        ],
       ...,
       [ 0.4658772 , 11.261055  ,  0.27667668],
       [ 0.49328175, 11.2655325 ,  0.2765748 ],
       [ 0.4658772 , 11.2655325 ,  0.2765748 ]], dtype=float32)

In [20]:
x, y, z = np.split(fc1, [1, 2], axis=1)


array([[0.        ],
       [0.        ],
       [0.49328175],
       ...,
       [0.4658772 ],
       [0.49328175],
       [0.4658772 ]], dtype=float32)